#task #1: 問題提起とビジネスケースの理解

#task #2: データセットとライブラリのインポート

In [ ]:
# Google ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow import keras


In [ ]:
# 株価データの読み込み
stock_price_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Python & ML in Finance/Part 3. AI and ML in Finance/stock.csv')
stock_price_df

In [ ]:
# 株式のボリュームデータを読み込む
stock_vol_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Python & ML in Finance/Part 3. AI and ML in Finance/stock_volume.csv")
stock_vol_df

In [ ]:
# 日付を基準にデータをソート
stock_price_df = stock_price_df.sort_values(by = ['Date'])
stock_price_df

In [ ]:
# 日付を基準にしてデータをソート
stock_vol_df = stock_vol_df.sort_values(by = ['Date'])
stock_vol_df

In [ ]:
# 株価データにNull値が存在するかどうかを調べる
stock_price_df.isnull().sum()

In [ ]:
# 株式のボリュームデータにNull値が存在するかどうかの確認
stock_vol_df.isnull().sum()

In [ ]:
# 株価データフレーム情報の取得
stock_price_df.info()

In [ ]:
# 株価ボリュームデータフレーム情報の取得
stock_vol_df.info()

In [ ]:
stock_vol_df.describe()

**ミニチャレンジ#1の解答:**
- Apple株の平均取引量はどのくらいですか？
- **sp500の最大取引量はどのくらいですか**?
- **最も取引されている証券はどれですか？ 答えについてコメントしてください**。
- **指定された期間におけるS&P500の平均株価はいくらですか？
- **テスラ株の最高株価はいくらですか**?
#

#task #3: 探索的データ分析と可視化の実行

In [ ]:
# 株価を初値に基づいて正規化する関数
def normalize(df):
  x = df.copy()
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x

In [ ]:
# Plotly Expressを使ってインタラクティブなプロットをする関数
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
# 株式データのインタラクティブ・チャートを描画する
interactive_plot(stock_price_df, 'Stock Prices')

**ミニチャレンジ #2:**
- **全銘柄の出来高データセットをプロットし、見えてきたオブザベーションを列挙する**。
- **正規化された株価と出来高のデータセットをプロットしてください**。


# task #4: ai/mlモデルの学習前にデータを準備する

![alt text](https://drive.google.com/uc?id=1uXYYHfgeJyncu4BZRAooTC4iCclH9e9B)

In [ ]:
# 日付、株価、出来高を1つのデータフレームに連結する関数
def individual_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

In [ ]:
# AI/MLモデルの入出力(ターゲット)データを返す関数
# 将来の株価を予測することが目的であることに注意してください 
# 今日の目標株価が明日の株価になる 

def trading_window(data):
  
# 1日後の価格を含む列を作る
  n = 1

# 日付、株価、出来高を1つのデータフレームに連結する関数
  data['Target'] = data[['Close']].shift(-n)
  
# 新しいデータセットを返す 
  return data

In [ ]:
# 関数をテストして、AAPLの個別の株価と出来高を取得してみよう
price_volume_df = individual_stock(stock_price_df, stock_vol_df, 'AAPL')
price_volume_df

In [ ]:
price_volume_target_df = trading_window(price_volume_df)
price_volume_target_df

In [ ]:
# 最後の行はNULL値になるので削除する
price_volume_target_df = price_volume_target_df[:-1]
price_volume_target_df

In [ ]:
# データのスケーリング
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
price_volume_target_scaled_df = sc.fit_transform(price_volume_target_df.drop(columns = ['Date']))

In [ ]:
price_volume_target_scaled_df

In [ ]:
price_volume_target_scaled_df.shape

In [ ]:
# フィーチャーとターゲットの作成
X = price_volume_target_scaled_df[:,:2]
y = price_volume_target_scaled_df[:,2:]

In [ ]:
# dataframeを配列に変換する
# X = np.asarray(X)
# y = np.asarray(y)

X.shape, y.shape

In [ ]:
# 時系列では順序が重要なので、このようにデータを分割する
# train test splitはデータがシャッフルされてしまうので、デフォルトの設定では使用しないことに注意してください。

split = int(0.65 * len(X))
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
# データのプロット機能を定義する
def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

show_plot(X_train, 'Training Data')
show_plot(X_test, 'Testing Data')


**ミニチャレンジ#3:**
- **作成したパイプラインをS&P500とAmazonのデータセットでテストする**。


# task #7: リッジ線形回帰モデルの構築と学習

In [ ]:
from sklearn.linear_model import Ridge
# Ridge回帰はL2正則化を用いた線形最小二乗を行うことに注意。
# リッジ線形回帰モデルの作成と学習
regression_model = Ridge()
regression_model.fit(X_train, y_train)

In [ ]:
# モデルをテストし、その精度を計算する 
lr_accuracy = regression_model.score(X_test, y_test)
print("Linear Regression Score: ", lr_accuracy)

In [ ]:
# 予測を行う
predicted_prices = regression_model.predict(X)
predicted_prices

In [ ]:
# 予測された値をリストに追加
Predicted = []
for i in predicted_prices:
  Predicted.append(i[0])

In [ ]:
len(Predicted)

In [ ]:
# クローズの値をリストに追加する
close = []
for i in price_volume_target_scaled_df:
  close.append(i[0])


In [ ]:
# クローズ値をデータフレームに追加する
df_predicted = price_volume_target_df[['Date']]
df_predicted

In [ ]:
# 予測値をデータフレームに追加する
df_predicted['Close'] = close
df_predicted

In [ ]:
# 結果をプロットする
df_predicted['Prediction'] = Predicted
df_predicted

In [ ]:
**ミニチャレンジ #4:**
- **アルファに対する様々な正則化の値を使って実験する**。
- **αを増やすとどのような影響があるか**。
- **注：αのデフォルト値は=1**です。

interactive_plot(df_predicted, "Original Vs. Prediction")

# task #8: ニューラルネットワークの背後にある理論と直観を理解する

![alt text](https://drive.google.com/uc?id=1U2auh7KSalF4qc8iWFScawEB3TniFeXw)

![alt text](https://drive.google.com/uc?id=1xsy74Dj9JyuvwcPzmQcwN9hzHGevAvKB)

![alt text](https://drive.google.com/uc?id=1b2wIRl63jXgyoh_w1ysxk3XvTJ5j8864)

# task #9: 芸術的なニューラルネットワークがどのように学習するかを理解する 

![alt text](https://drive.google.com/uc?id=1pg1rxBoHDtRNVNN0TyMvnCMVKw9ni5pN)

![alt text](https://drive.google.com/uc?id=1wc4NOBLC55Hb2s8s_hLI2LWykV4WmLwU)

![alt text](https://drive.google.com/uc?id=1zI3MbDUAws-gpPisqDrVL8BoupWRO6KJ)

![alt text](https://drive.google.com/uc?id=1twCXUdFtdFMGsicoQOWI0a9RiHMZ9MK_)

# task #10: リカレントニューラルネットワークの理論と直感を理解する

![alt text](https://drive.google.com/uc?id=1a6w38XX_W2Zcsvde5FDepUax1aRyvnBT)

![alt text](https://drive.google.com/uc?id=1tc1SRSaFp-zuvujAQY66E_JEoY6doI0m)

![alt text](https://drive.google.com/uc?id=10xmgZjv2IjaY4xFm81rwgKqOC0119tug)

![alt text](https://drive.google.com/uc?id=1CwBOagYD82QaayeGDndRjb_Irm5HIwBs)

![alt text](https://drive.google.com/uc?id=148mslyqerZ_gLzQIkPFiDsewCO5nMNK7)

![alt text](https://drive.google.com/uc?id=1_Ky5ek9gq_Ov6VLTkXFmmzEhvjo91tpB)

# TASK #11: 長短記憶ネットワークの理論と直感を理解する

![alt text](https://drive.google.com/uc?id=1cd3jZpTuSzsPjMps6SRFqvXNfs4jA_vu)

![alt text](https://drive.google.com/uc?id=1URS5Ny54g-c3ookAL1FLn28TYFlbDIdG)

![alt text](https://drive.google.com/uc?id=1Cm3Unnn_b71rfklW_ejm50xYOYkXh8cl)

![alt text](https://drive.google.com/uc?id=1pU8lbF965n38ZspjIFsEAU3CEjpclpfd)

![alt text](https://drive.google.com/uc?id=1zSqTnPZDfiz6qRgMHaFB9wpB55Fnzwf4)

# task #12: lstm 時系列モデルの学習

In [ ]:
# 関数をテストして、AAPLの個別株価と出来高を取得してみましょう。
price_volume_df = individual_stock(stock_price_df, stock_vol_df, 'sp500')
price_volume_df

In [ ]:
# 終値と出来高のデータを学習データとして取得 (Input)
training_data = price_volume_df.iloc[:, 1:3].values
training_data

In [ ]:
# データの正規化
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_data)

In [ ]:
# トレーニングデータとテストデータを作成します．トレーニングデータには，今日と前日の値が含まれます．
X = []
y = []
for i in range(1, len(price_volume_df)):
    X.append(training_set_scaled [i-1:i, 0])
    y.append(training_set_scaled [i, 0])

In [ ]:
X

In [ ]:
# データを配列形式に変換
X = np.asarray(X)
y = np.asarray(y)

In [ ]:
# データの分割
split = int(0.7 * len(X))
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

In [ ]:
# 1次元の配列を3次元の配列に整形してモデルに入力する
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_train.shape, X_test.shape

In [ ]:
# モデルの作成
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(150, return_sequences= True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150, return_sequences=True)(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss="mse")
model.summary()

In [ ]:
# モデルのトレーシング
history = model.fit(
    X_train, y_train,
    epochs = 20,
    batch_size = 32,
    validation_split = 0.2
)

In [ ]:
# 予測を行う
predicted = model.predict(X)

In [ ]:
# 予測された値をリストに追加
test_predicted = []

for i in predicted:
  test_predicted.append(i[0][0])

In [ ]:
test_predicted

[0.0119830705,
 0.008988192,
 0.011127962,
 0.017845431,
 0.020867169,
 0.021278897,
 0.02156899,
 0.020937359,
 0.026277523,
 0.022706093,
 0.021723373,
 0.020170104,
 0.019889425,
 0.025355209,
 0.026034059,
 0.035103083,
 0.034835894,
 0.03611089,
 0.037475087,
 0.038408153,
 0.03404386,
 0.038323775,
 0.037728295,
 0.034320354,
 0.041264344,
 0.042760804,
 0.043220587,
 0.0410861,
 0.043807097,
 0.04487701,
 0.045745216,
 0.047899786,
 0.04484888,
 0.048796467,
 0.046702735,
 0.044215325,
 0.034381278,
 0.038727045,
 0.044956807,
 0.047284797,
 0.047388054,
 0.059066363,
 0.058281377,
 0.062193077,
 0.06293148,
 0.06555619,
 0.06356642,
 0.062329486,
 0.057576336,
 0.0596117,
 0.06873708,
 0.06685947,
 0.06357582,
 0.0625129,
 0.06495402,
 0.06992787,
 0.067264125,
 0.060481466,
 0.060067724,
 0.052605078,
 0.041522313,
 0.04627088,
 0.055127896,
 0.046998456,
 0.04667455,
 0.056636356,
 0.05398622,
 0.050125327,
 0.050881386,
 0.045440145,
 0.047801163,
 0.05659402,
 0.06096107,
 

In [ ]:
df_predicted = price_volume_df[1:][['Date']]
df_predicted

,Date
1,2012-01-13
2,2012-01-17
3,2012-01-18
4,2012-01-19
5,2012-01-20
...,...
2154,2020-08-05
2155,2020-08-06
2156,2020-08-07
2157,2020-08-10


In [ ]:
df_predicted['predictions'] = test_predicted

In [ ]:
df_predicted

,Date,predictions
1,2012-01-13,0.011983
2,2012-01-17,0.008988
3,2012-01-18,0.011128
4,2012-01-19,0.017845
5,2012-01-20,0.020867
...,...,...
2154,2020-08-05,0.972882
2155,2020-08-06,0.982849
2156,2020-08-07,0.992864
2157,2020-08-10,0.993856


In [ ]:
# データをプロットする
close = []
for i in training_set_scaled:
  close.append(i[0])


In [ ]:
df_predicted['Close'] = close[1:]

In [ ]:
df_predicted

,Date,predictions,Close
1,2012-01-13,0.011983,0.005242
2,2012-01-17,0.008988,0.007414
3,2012-01-18,0.011128,0.014231
4,2012-01-19,0.017845,0.017295
5,2012-01-20,0.020867,0.017713
...,...,...,...
2154,2020-08-05,0.972882,0.972307
2155,2020-08-06,0.982849,0.982453
2156,2020-08-07,0.992864,0.983459
2157,2020-08-10,0.993856,0.987819


In [ ]:
# データをプロットする
interactive_plot(df_predicted, "Original Vs Prediction")

**ミニチャレンジ #5:**
- パイプラインを少なくとも3つの他の銘柄でテストする**。
- **LSTMモデルのパラメータを変えて実験する(例：50ユニットの代わりに150ユニットを使う)、モデルのサマリーを出力し、モデルを再学習する**。


# **ミニ・チャレンジの解決策 **
**ミニチャレンジ#1の解答:**
- Apple株の平均取引量はどのくらいですか？
- **sp500の最大取引量はどのくらいですか**?
- **最も取引されている証券はどれですか？ 答えについてコメントしてください**。
- **指定された期間におけるS&P500の平均株価はいくらですか？
- **テスラ株の最高株価はいくらですか**?


In [ ]:
# 株式の出来高データフレームの統計データを取得します。
# Apple株の平均取引量は2.498238e+06	
# S&P500の平均出来高は3.680732e+09

# なぜS&P500が最も取引されているのか？こちらの記事をご覧ください。
# https://www.investopedia.com/articles/personal-finance/022216/put-10000-sp-500-etf-and-wait-20-years.asp

# 上の記事からの引用です。
# "S&P500指数は、米国の株式市場で取引されている大企業を対象とした幅広い指標である。長期間にわたって 
# 長期的には、インデックスをパッシブに保有した方が、アクティブに取引したり、単一の銘柄を選んだりするよりも、良い結果が得られることが多い。
# 長い時間軸で見ると、インデックスは通常、積極的に管理されたポートフォリオよりも良いリターンを生み出します。"

stock_vol_df.describe()

In [ ]:
# 価格データフレームの統計データを得る
stock_price_df.describe()

In [ ]:
# Average price for S&P500 = 2218.749554
# Maximum Tesla Price = 1643.000000

**ミニチャレンジ#2の解答:**
- **正規化された株価と出来高のデータセットをプロットします**。


In [ ]:
# Plot interactive chart for volume data.
# S&P500 の取引は個別銘柄に比べて桁違いであることに注目してほしい

interactive_plot(stock_vol_df, 'Stocks Volume')

In [ ]:
# 正規化された株価データのインタラクティブ・チャートをプロットする
interactive_plot(normalize(stock_price_df), 'Stock Prices')

# データを正規化して, 出来高データのインタラクティブ・チャートを再プロットしましょう
interactive_plot(normalize(stock_vol_df), 'Normalized Volume')

**ミニチャレンジ #3 解決策:**
- **AAPLの代わりにS&P500とAMZNのデータセットでパイプラインをテストする**。


In [ ]:
# 関数をテストして、S&P500の個別銘柄の価格とボリュームを取得してみましょう。
price_volume_df = individual_stock(stock_price_df, stock_vol_df, 'sp500')
価格_ボリューム_df

# 関数をテストして、Amazonの個別株の価格と出来高を取得してみましょう 
price_volume_df = individual_stock(stock_price_df, stock_vol_df, 'AMZN')
price_volume_df

**ミニチャレンジ#4の解決策：***。
- **αの正則化の値をいろいろ変えて実験してみる**。
- **αを増加させるとどのような影響があるか**。
- **注：αのデフォルト値は=1です**。


In [ ]:
from sklearn.linear_model import Ridge
# Ridge回帰はL2正則化で線形最小二乗を行うことに注意してください。
# Ridge線形回帰モデルの作成と学習

regression_model = Ridge(alpha = 2)
regression_model.fit(X_train, y_train)

**ミニチャレンジ #5 解決策:**
- パイプラインを少なくとも3つの他の銘柄でテストする**。
- **様々なLSTMモデルのパラメータで実験し(例：50ユニットの代わりに150ユニットを使用)、モデルのサマリーをプリントアウトし、モデルを再学習する**。



In [ ]:
# モデルの作成
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(150, return_sequences= True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150, return_sequences=True)(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss="mse")
model.summary()